lab work №1.2

Student name - Khrystyna

Student surname - Zyryanova

Group - CS-32

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

STUDENT_NO = 11

# 1. Завантаження датасету California Housing
housing = fetch_california_housing(as_frame=True)
df = housing.frame
X = df.drop('MedHouseVal', axis=1) # Ознаки (Features)
y = df['MedHouseVal']              # Цільова змінна (Target)

print(f"Розмірність даних: {df.shape}")
print(f"Перші 5 рядків даних:")
print(df.head())

# 2. Розділення даних на тренувальний та тестовий набори
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=STUDENT_NO
)

print(f"\nРозмір тренувального набору: {X_train.shape}")

# 3. Масштабування даних
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Перетворення назад у DataFrame
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

print("\nМасштабування завершено.")
print(f"Середнє значення MedInc після масштабування (має бути близьке до 0): {X_train_scaled_df['MedInc'].mean():.4f}")

Розмірність даних: (20640, 9)
Перші 5 рядків даних:
   MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0  8.3252      41.0  6.984127   1.023810       322.0  2.555556     37.88   
1  8.3014      21.0  6.238137   0.971880      2401.0  2.109842     37.86   
2  7.2574      52.0  8.288136   1.073446       496.0  2.802260     37.85   
3  5.6431      52.0  5.817352   1.073059       558.0  2.547945     37.85   
4  3.8462      52.0  6.281853   1.081081       565.0  2.181467     37.85   

   Longitude  MedHouseVal  
0    -122.23        4.526  
1    -122.22        3.585  
2    -122.24        3.521  
3    -122.25        3.413  
4    -122.25        3.422  

Розмір тренувального набору: (16512, 8)

Масштабування завершено.
Середнє значення MedInc після масштабування (має бути близьке до 0): 0.0000


In [3]:
param_grid_rfr = {
    'n_estimators': [50, 100], 
    'max_depth': [15, 25],   
    'min_samples_split': [5, 10]
}
rfr = RandomForestRegressor(random_state=STUDENT_NO)
grid_search_rfr = GridSearchCV(estimator=rfr, param_grid=param_grid_rfr, scoring='neg_mean_squared_error', cv=3, n_jobs=-1, verbose=1)

print("Запуск GridSearchCV для RandomForestRegressor...")
grid_search_rfr.fit(X_train_scaled_df, y_train)

best_rfr = grid_search_rfr.best_estimator_
print("\n--- Результати Налаштування RFR ---")
print(f"Найкращі параметри: {grid_search_rfr.best_params_}")
print(f"Найкращий результат (Negative MSE): {grid_search_rfr.best_score_:.4f}")

🚀 Запуск GridSearchCV для RandomForestRegressor...
Fitting 3 folds for each of 8 candidates, totalling 24 fits

--- ✅ Результати Налаштування RFR ---
Найкращі параметри: {'max_depth': 25, 'min_samples_split': 5, 'n_estimators': 100}
Найкращий результат (Negative MSE): -0.2669


In [9]:
param_grid_gbr = {
    'n_estimators': [100, 300],
    'learning_rate': [0.05, 0.1], 
    'max_depth': [3, 5]
}
gbr = GradientBoostingRegressor(random_state=STUDENT_NO)
grid_search_gbr = GridSearchCV(estimator=gbr, param_grid=param_grid_gbr, scoring='neg_mean_squared_error', cv=3, n_jobs=-1, verbose=1)

print("\n Запуск GridSearchCV для GradientBoostingRegressor...")
grid_search_gbr.fit(X_train_scaled_df, y_train)

best_gbr = grid_search_gbr.best_estimator_
print("\n--- Результати Налаштування GBR ---")
print(f"Найкращі параметри: {grid_search_gbr.best_params_}")
print(f"Найкращий результат (Negative MSE): {grid_search_gbr.best_score_:.4f}")


 Запуск GridSearchCV для GradientBoostingRegressor...
Fitting 3 folds for each of 8 candidates, totalling 24 fits

--- Результати Налаштування GBR ---
Найкращі параметри: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300}
Найкращий результат (Negative MSE): -0.2244


In [10]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# --- Допоміжна функція (evaluate_regression_model) ---
def evaluate_regression_model(y_true, X, model, print_results=True):
    if hasattr(model, 'predict'):
        y_pred = model.predict(X)
    else:
        raise TypeError("Model type not supported. Please provide a scikit-learn model.")

    metrics = {
        'MAE': mean_absolute_error(y_true, y_pred),
        'MSE': mean_squared_error(y_true, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'R2': r2_score(y_true, y_pred)
    }

    if print_results:
        print("----- Model Evaluation -----")
        print(pd.Series(metrics).to_string(float_format="%.4f"))
        print("-----------------------------")

    return metrics

In [11]:
param_grid_svr = {
    'C': [1, 10], 
    'gamma': [0.1, 1], 
    'kernel': ['rbf'] 
} 
svr = SVR()
grid_search_svr = GridSearchCV(estimator=svr, param_grid=param_grid_svr, scoring='neg_mean_squared_error', cv=3, n_jobs=-1, verbose=1)

print("\n Запуск GridSearchCV для SVR...")
grid_search_svr.fit(X_train_scaled_df, y_train)

best_svr = grid_search_svr.best_estimator_
print("\n--- Результати Налаштування SVR ---")
print(f"Найкращі параметри: {grid_search_svr.best_params_}")
print(f"Найкращий результат (Negative MSE): {grid_search_svr.best_score_:.4f}")


 Запуск GridSearchCV для SVR...
Fitting 3 folds for each of 4 candidates, totalling 12 fits

--- Результати Налаштування SVR ---
Найкращі параметри: {'C': 10, 'gamma': 1, 'kernel': 'rbf'}
Найкращий результат (Negative MSE): -0.3330


In [12]:
final_models = {
    "RandomForestRegressor": best_rfr,
    "GradientBoostingRegressor": best_gbr,
    "SVR": best_svr
}

results = {}

print("\n=== Фінальне Тестування на X_test ===")

X_test_final = X_test_scaled_df 

for name, model in final_models.items():
    # evaluate_regression_model
    metrics = evaluate_regression_model(y_test, X_test_final, model=model, print_results=False)
    
    print(f"\n--- {name} ---")
    print(f"R2 на тесті: {metrics['R2']:.4f}")
    print(f"RMSE на тесті: {metrics['RMSE']:.4f}")
    
    results[name] = metrics

# Фінальне порівняння та визначення переможця
if results:
    best_r2 = max(metrics['R2'] for metrics in results.values())
    best_model_name = [name for name, metrics in results.items() if metrics['R2'] == best_r2][0]

    print("\n==============================================")
    print(f"Найкраща модель на тестовому наборі: {best_model_name}")
    print(f"R2 Score: {best_r2:.4f}")
    print("==============================================")


=== Фінальне Тестування на X_test ===

--- RandomForestRegressor ---
R2 на тесті: 0.8120
RMSE на тесті: 0.5030

--- GradientBoostingRegressor ---
R2 на тесті: 0.8395
RMSE на тесті: 0.4648

--- SVR ---
R2 на тесті: 0.7672
RMSE на тесті: 0.5597

Найкраща модель на тестовому наборі: GradientBoostingRegressor
R2 Score: 0.8395
